In [124]:
# Import necessary modules
import requests
from bs4 import BeautifulSoup
import numpy as np
import re

#Make log
import scraping_class
logfile = 'log_jobindex_scraping.txt'
connector = scraping_class.Connector(logfile)

#Define todays date
from datetime import date
today = date.today().strftime("%Y%m%d") # to get format 20080101

import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import slate3k as slate
import pandas as pd

#pip install certifi
#pip install urllib3[secure]
import certifi
import urllib3
http = urllib3.PoolManager()

## Total number of daily postings 2015-2020

In [ ]:

import pandas as pd
from datetime import datetime
datelist = pd.date_range(start='01/01/2015', end='26/08/2020', freq='M').date.tolist()
datelist = [str(date).strip("()").strip("datetime.date").replace(',','').replace('-','') for date in datelist]

date_totals = []
for date in datelist:
    url = f'https://www.jobindex.dk/jobsoegning?maxdate={date}&mindate={date}&jobage=archive'
    response = requests.get(url) 
    soup = BeautifulSoup(response.text,'lxml')
    search_results = BeautifulSoup(str(soup.find_all("div", {'class':'jix_pagination_total'}))).get_text()
    total = re.findall(r'af(?s)(.*)resultat',str(search_results))
    total = str(total).strip("[]").strip("''").strip().replace('.','')
    if total == 0:
            continue
    #Use total number of job postings to calculate total number of pages
    total=int(total)
    t_d = [date,total]
    print(date)
    date_totals.append(t_d)
    
totals = pd.DataFrame(date_totals)
totals.columns = ['Date','Postings Published']
total.to_pickle(r'C:\Users\lisbe\OneDrive\11. Semester\Social Data Science\Modules\Total_jobs15-20')

In [89]:
import re
import requests
from datetime import date
from bs4 import BeautifulSoup
import numpy as np

#Define the basic url
url = 'https://www.jobindex.dk/jobsoegning'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html,'lxml')

#Areas are given in the html by div id=areas and class=area_label
areas_html = soup.find('div', {'id':'areas'})#('a', {'class':'area_label'})
areas_div = str(areas_html).split('href="/jobsoegning/')[1:]

#Find the names of each of the areas
areas = ()
areas_delete = ('skaane', 'faeroeerne', 'udlandet', 'groenland', 'danmark') #delete regions outside Denmark's borders
for area in areas_div:
    link = area.split('"')[0]
    if link in areas_delete:
        del link
    else:
        areas += (link,)
areas += ('danmark',) #make sure Danmark is the last element in the tuple, since it is ordered. 
#print(areas)

#Categories are given in the html by and id=categories
cat_html = soup.find('div', {'id':'categories'})
cat_div = soup.find_all('a',{'class':'filter-section-header title collapsed'}) #the filter collapses

categories = []
subcategories = []
for cat in cat_div:
    #first find the overall categories
    cat_name = re.findall(r'(?<=<span>)(.*)(?=<span class)', str(cat))[0]
    cat_name = cat_name.replace('\xad','')
    cat_links = re.findall("href=[\"\'](.*?)[\"\']", str(cat))
    cat_id = str(cat_links[0])
    cat_id = cat_id.replace('#','')
    category = [cat_name,cat_id]
    categories.append(category)
    #next find the sub-categories
    subcat_div = str(cat_html).split('href="/jobsoegning/')[1:]
for subcat in subcat_div:
    subcategory = subcat.split('"')[0]
    #cat_id = re.findall('(.*?)/', str(subcategory))
    #subcat_done = [subcategory,cat_id]
    subcategories.append(subcategory)
#print(categories)
#print(subcategories)
    

## TOTAL NUMBER OF MONTHLY POSTINGS PER AREA

In [125]:
import pandas as pd
from datetime import datetime

start_date= pd.date_range('2015-01-01','2020-09-01', freq='1M')-pd.offsets.MonthBegin(1)
start_date = [str(date).strip("()").strip("datetime.date").replace(',','').replace('-','').replace(':','').strip(' 000000') for date in start_date]

end_date = pd.date_range('2015-02-01','2020-10-01', freq='1M')-pd.offsets.MonthEnd(1)
end_date = [str(date).strip("()").strip("datetime.date").replace(',','').replace('-','').replace(':','').strip('000000').rstrip() for date in end_date]

def merge(list1, list2): 
    merged_list = tuple(zip(list1, list2))  
    return merged_list 

month_dates = merge(start_date,end_date)

date_totals_area = []
for area in areas:
    for date in month_dates:
        url = f'https://www.jobindex.dk/jobsoegning/{area}?jobage=archive&maxdate={date[1]}&mindate={date[0]}'
        response = requests.get(url) 
        soup = BeautifulSoup(response.text,'lxml')
        search_results = BeautifulSoup(str(soup.find_all("div", {'class':'jix_pagination_total'}))).get_text()
        total = re.findall(r'af(?s)(.*)resultat',str(search_results))
        total = str(total).strip("[]").strip("''").strip().replace('.','')
        if total == 0:
                continue
        #Use total number of job postings to calculate total number of pages
        total=int(total)
        t_d = [date,total,area]
        print(date)
        date_totals_area.append(t_d)
        
totals_area = pd.DataFrame(date_totals_area)
totals_area.columns = ['Date','Postings Published','Area']
totals_area.to_pickle(r'C:\Users\lisbe\OneDrive\11. Semester\Social Data Science\Modules\Total_jobs15-20_areas')

('20150101', '20150131')
('20150201', '20150228')
('20150301', '20150331')
('20150401', '20150430')
('20150501', '20150531')
('20150601', '20150630')
('20150701', '20150731')
('20150801', '20150831')
('20150901', '20150930')
('20151001', '20151031')
('20151101', '20151130')
('20151201', '20151231')
('20160101', '20160131')
('20160201', '20160229')
('20160301', '20160331')
('20160401', '20160430')
('20160501', '20160531')
('20160601', '20160630')
('20160701', '20160731')
('20160801', '20160831')
('20160901', '20160930')
('20161001', '20161031')
('20161101', '20161130')
('20161201', '20161231')
('20170101', '20170131')
('20170201', '20170228')
('20170301', '20170331')
('20170401', '20170430')
('20170501', '20170531')
('20170601', '20170630')
('20170701', '20170731')
('20170801', '20170831')
('20170901', '20170930')
('20171001', '20171031')
('20171101', '20171130')
('20171201', '20171231')
('20180101', '20180131')
('20180201', '20180228')
('20180301', '20180331')
('20180401', '20180430')


('20190901', '20190930')
('20191001', '20191031')
('20191101', '20191130')
('20191201', '20191231')
('20200101', '20200131')
('20200201', '20200229')
('20200301', '20200331')
('20200401', '20200430')
('20200501', '20200531')
('20200601', '20200630')
('20200701', '20200731')
('20200801', '20200831')
('20150101', '20150131')
('20150201', '20150228')
('20150301', '20150331')
('20150401', '20150430')
('20150501', '20150531')
('20150601', '20150630')
('20150701', '20150731')
('20150801', '20150831')
('20150901', '20150930')
('20151001', '20151031')
('20151101', '20151130')
('20151201', '20151231')
('20160101', '20160131')
('20160201', '20160229')
('20160301', '20160331')
('20160401', '20160430')
('20160501', '20160531')
('20160601', '20160630')
('20160701', '20160731')
('20160801', '20160831')
('20160901', '20160930')
('20161001', '20161031')
('20161101', '20161130')
('20161201', '20161231')
('20170101', '20170131')
('20170201', '20170228')
('20170301', '20170331')
('20170401', '20170430')


In [130]:
len(totals_area)

612

## TOTAL NUMBER OF MONTHLY POSTINGS PER CATEGORY

In [131]:
date_totals_subcat = []
for subcategory in subcategories:
    for date in month_dates:
        url = f'https://www.jobindex.dk/jobsoegning/{subcategory}?jobage=archive&maxdate={date[1]}&mindate={date[0]}'
        response = requests.get(url) 
        soup = BeautifulSoup(response.text,'lxml')
        search_results = BeautifulSoup(str(soup.find_all("div", {'class':'jix_pagination_total'}))).get_text()
        total = re.findall(r'af(?s)(.*)resultat',str(search_results))
        total = str(total).strip("[]").strip("''").strip().replace('.','')
        if total == 0:
                continue
        #Use total number of job postings to calculate total number of pages
        total=int(total)
        t_d = [date,total,category]
        print(date)
        date_totals_subcat.append(t_d)
        
totals_subcat = pd.DataFrame(date_totals_subcat)
totals_subcat.columns = ['Date','Postings Published','Subcategory']
totals_subcat.to_pickle(r'C:\Users\lisbe\OneDrive\11. Semester\Social Data Science\Modules\Total_jobs15-20subcategories')

SSLError: HTTPSConnectionPool(host='www.jobindex.dk', port=443): Max retries exceeded with url: /jobsoegning/it/database?jobage=archive&maxdate=20150131&mindate=20150101 (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))

In [ ]:
# TOTAL POSTINGS MONTHLY PER AREA PER CATEGORY

date_totals_area_subcat = []
for area in areas:
    for category in categories:
        for date in month_dates:
            url = f'https://www.jobindex.dk/jobsoegning?maxdate={date[1]}&mindate={date[0]}&jobage=archive'
            response = requests.get(url) 
            soup = BeautifulSoup(response.text,'lxml')
            search_results = BeautifulSoup(str(soup.find_all("div", {'class':'jix_pagination_total'}))).get_text()
            total = re.findall(r'af(?s)(.*)resultat',str(search_results))
            total = str(total).strip("[]").strip("''").strip().replace('.','')
            if total == 0:
                    continue
            #Use total number of job postings to calculate total number of pages
            total=int(total)
            t_d = [date,total,subcategory,area]
            print(date)
            date_totals_area_cat.append(t_d)

totals_cat_area = pd.DataFrame(date_totals_area_cat)
totals_cat_area.columns = ['Date','Postings Published','Category','Area']
totals_cat_area.to_pickle(r'C:\Users\lisbe\OneDrive\11. Semester\Social Data Science\Modules\Total_jobs15-20_categories_areas')